In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import petab.v1.C as C
import pypesto
import pypesto.optimize as optimize
import pypesto.petab
import pypesto.sample as sample
import pypesto.visualize as visualize

# from polypesto._petab import CRP2_CPE as crp
from polypesto.core.params import ParameterContainer
from polypesto.core.pypesto import create_problem_set, load_pypesto_problem
from polypesto.utils.plot import plot_all_measurements

from polypesto.models.sbml import CRP2_CPE
from experiments.CRP2_CPE.problem import ds_0, ds_1

%load_ext autoreload
%autoreload 2

In [ ]:
pc = ParameterContainer.from_json("/PolyPESTO/experiments/CRP2_CPE/params.json")
pg = pc.get_parameter_group("REVERSIBLE")

paths = create_problem_set(CRP2_CPE, pg, ds_1(), force_compile=False)

In [ ]:
p_id = "p5"
# yaml_path = paths[p_id]
yaml_path = "/PolyPESTO/src/data/fitting/CRP2_CPE/petab/p5/petab.yaml"
importer, problem = load_pypesto_problem(yaml_path, "CRP2_CPE", force_compile=True)

In [ ]:
plot_all_measurements(
    importer.petab_problem.measurement_df,
    # group_by=C.SIMULATION_CONDITION_ID,
    group_by=C.OBSERVABLE_ID,
    format_axes_kwargs={
        "set_xlabel": "Total Conversion",
        "set_ylabel": "Monomer Conversion",
        # "set_xlim": (0, 1),
        # "set_ylim": (0, 1),
    },
    plot_style="both",
    alpha=0.5,
)
plt.show()

In [ ]:
optimizer = optimize.ScipyOptimizer()  # method="Nelder-Mead")
history_options = pypesto.HistoryOptions(trace_record=True)
engine = pypesto.engine.MultiProcessEngine()
n_starts = 1

# run optimization of problem 1
import amici

# problem.objective.amici_solver.setSensitivityMethod(amici.SensitivityMethod.forward)
problem.objective.amici_solver.setSensitivityOrder(amici.SensitivityOrder.none)
problem.objective.amici_solver.setMaxStepSize(1e-1)
problem.objective.amici_model.setAlwaysCheckFinite(True)
# problem.objective.amici_solver.setSensitivityOrder(1)
# problem.objective.amici_solver.
result = optimize.minimize(
    problem=problem,
    optimizer=optimizer,
    n_starts=n_starts,
    engine=engine,
    history_options=history_options,
)

In [ ]:
sampler = sample.AdaptiveParallelTemperingSampler(
    internal_sampler=sample.AdaptiveMetropolisSampler(),
    n_chains=3,
)

n_samples = 10000
result = sample.sample(
    problem,
    n_samples=n_samples,
    sampler=sampler,
    x0=np.array([1, 1, 0, 0]),
    filename=None,
)
elapsed_time = result.sample_result.time
print(f"Elapsed time: {round(elapsed_time,2)}")

In [ ]:
ax = visualize.sampling_parameter_traces(result, use_problem_bounds=True, size=(12, 5))

p = pg.by_id("p5").to_dict()
ax[0][0].plot([0, n_samples], np.log10([p["rA"], p["rA"]]), "r--")
ax[0][1].plot([0, n_samples], np.log10([p["rB"], p["rB"]]), "r--")
# ax[1][0].plot([0, n_samples], [p["KAA"], p["KAA"]], "r--")
#  ax[1][1].plot([0, n_samples], [p["KBA"], p["KBA"]], "r--")